# DocGemma Connect - Colab Demo

Medical AI assistant powered by MedGemma 4B with structured generation.

**Requirements:**
- GPU runtime (T4 or better)
- HuggingFace account with MedGemma access
- GitHub Personal Access Token (for private repo)

**Setup:**
1. Go to **Runtime > Change runtime type > T4 GPU**
2. Click the **key icon** in the left sidebar to open Secrets
3. Add `GITHUB_TOKEN` with your GitHub PAT
4. Add `HF_TOKEN` with your HuggingFace token (optional, for auto-login)

## 1. Install DocGemma from GitHub (Private Repo)

Since this is a private repository, you need a GitHub Personal Access Token (PAT).

**Create a PAT:** GitHub → Settings → Developer settings → Personal access tokens → Generate new token (classic) → Select `repo` scope

In [ ]:
# Option 1: Use Colab Secrets (recommended)
# Add your GitHub PAT as a secret named "GITHUB_TOKEN" in the Colab sidebar (key icon)
from google.colab import userdata
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
!pip install -q git+https://{GITHUB_TOKEN}@github.com/galinilin/docgemma-connect.git

# Option 2: Direct token (less secure - token visible in notebook)
# !pip install -q git+https://ghp_yourtoken@github.com/galinilin/docgemma-connect.git

## 2. Authenticate with HuggingFace

MedGemma requires approval. Request access at: https://huggingface.co/google/medgemma-1.5-4b-it

In [ ]:
from huggingface_hub import login
from google.colab import userdata

# Option 1: Use Colab Secrets (if you added HF_TOKEN)
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN)
    print("Logged in via Colab secret")
except:
    # Option 2: Interactive login (will prompt for token)
    login()

## 3. Load the Model

In [ ]:
from docgemma import DocGemma

# Initialize and load the model
model = DocGemma()
model.load()

print(f"Model loaded on: {model.device}")
print(f"Using dtype: {model.dtype}")

## 4. Emergency Classification

Classify if a medical query is an emergency using constrained generation.

In [ ]:
# Test emergency classification
test_cases = [
    "I have a mild headache",
    "I'm experiencing severe chest pain and difficulty breathing",
    "My child has a fever of 38C",
    "I think I'm having a stroke, my face feels numb",
]

print("Emergency Classification Results:")
print("-" * 50)
for case in test_cases:
    result = model.classify_emergency(case)
    print(f"Input: {case}")
    print(f"Result: {result}")
    print()

## 5. User Type Classification

Detect if the user is a patient or medical expert based on their language.

In [ ]:
# Test user type classification
test_cases = [
    "My tummy hurts after eating",
    "Patient presents with acute epigastric pain, possible peptic ulcer",
    "What does my blood test result mean?",
    "Differential diagnosis for elevated troponin with normal ECG?",
]

print("User Type Classification Results:")
print("-" * 50)
for case in test_cases:
    result = model.classify_user_type(case)
    print(f"Input: {case}")
    print(f"Result: {result}")
    print()

## 6. Free-form Generation

Generate medical responses using the base model.

In [ ]:
# Generate a medical response
prompt = "What are the common causes of headaches and when should I see a doctor?"

response = model.generate(prompt, max_new_tokens=256)
print(f"Prompt: {prompt}")
print(f"\nResponse:\n{response}")

## 7. Structured Generation with Pydantic

Generate responses that conform to a specific schema using Outlines.

In [ ]:
from pydantic import BaseModel
from typing import Literal

# Define a structured output schema
class TriageResult(BaseModel):
    urgency: Literal["low", "medium", "high", "critical"]
    category: str
    recommendation: str

# Generate structured output
prompt = """Triage the following patient complaint and provide a structured assessment.

Patient complaint: I've had a persistent cough for 2 weeks with some yellow mucus.

Provide your assessment:"""

result = model.generate_outlines(prompt, TriageResult)
print(f"Urgency: {result.urgency}")
print(f"Category: {result.category}")
print(f"Recommendation: {result.recommendation}")

## 8. Combined Triage Pipeline Example

A simple example of the decision-tree approach.

In [ ]:
def triage_query(model, query: str) -> dict:
    """Simple triage pipeline using DocGemma's classification."""
    
    # Step 1: Check for emergency
    emergency_status = model.classify_emergency(query)
    
    if emergency_status == "emergency":
        return {
            "status": "EMERGENCY",
            "action": "Call emergency services immediately (911)",
            "query": query
        }
    
    # Step 2: Classify user type
    user_type = model.classify_user_type(query)
    
    # Step 3: Generate appropriate response
    if user_type == "patient":
        system_context = "Respond in simple, easy-to-understand language for a patient."
    else:
        system_context = "Respond with clinical terminology appropriate for a medical professional."
    
    full_prompt = f"{system_context}\n\nQuery: {query}"
    response = model.generate(full_prompt, max_new_tokens=200)
    
    return {
        "status": "non_emergency",
        "user_type": user_type,
        "response": response,
        "query": query
    }

# Test the pipeline
queries = [
    "I have sudden severe chest pain radiating to my arm",
    "What's the best way to treat a common cold?",
    "Recommended prophylaxis for DVT in post-operative patients?",
]

for query in queries:
    print("=" * 60)
    result = triage_query(model, query)
    for key, value in result.items():
        print(f"{key}: {value}")
    print()